# TFEX Series List Service

Learn how to fetch and analyze the complete list of futures and options contracts traded on TFEX (Thailand Futures Exchange).

## Overview

### What is TFEX?

TFEX (Thailand Futures Exchange) is Thailand's derivatives exchange where **futures** and **options** contracts are traded. These are financial instruments that derive their value from underlying assets like stock indices, commodities, or currencies.

### Futures vs Options

**Futures Contracts:**
- Agreement to buy/sell an asset at a predetermined price on a future date
- Both buyer and seller are **obligated** to fulfill the contract
- Example: S50Z25 (SET50 Index Futures expiring December 2025)
- Used for hedging risk or speculating on price movements

**Options Contracts:**
- Give the **right** (not obligation) to buy (Call) or sell (Put) an asset
- Buyer pays a premium for this right
- Seller keeps the premium even if option isn't exercised
- Example: S50Z25C900 (Call option on SET50 with 900 strike price)

### What This Service Does

The TFEX Series List Service allows you to:
- **Get all available contracts** - View every futures and options series trading on TFEX
- **Filter by type** - Separate futures from options, or filter by underlying asset
- **Track expiration dates** - Monitor when contracts expire (important for rollover planning)
- **Find active contracts** - Identify which series are currently trading

### When to Use This Service

Use this service when you need to:
- Research available derivatives contracts
- Plan contract rollovers (switching from expiring to next-month contracts)
- Analyze options chains for specific underlying assets
- Build automated trading or monitoring systems

## Prerequisites

### Installation

You need to install the `settfex` package:

```bash
pip install settfex
```

### Required Imports

This notebook uses:
- `settfex` - The main library for TFEX data
- `asyncio` - For asynchronous programming (don't worry, it's simple!)
- `datetime` - For working with dates and times

In [ ]:
# Install settfex (uncomment to run)
# !pip install settfex

In [ ]:
# Import required libraries
import asyncio
from datetime import datetime, timedelta

# Import TFEX services
from settfex.services.tfex import get_series_list

## Basic Usage

Let's start with the simplest use case: fetching the complete list of TFEX series.

In [ ]:
# Fetch all TFEX series
# Note: Jupyter notebooks handle async automatically in most modern versions
series_list = await get_series_list()

# Display basic statistics
print(f"Total Series: {series_list.count}")
print(f"Active Series: {len(series_list.filter_active_only())}")
print(f"Futures Contracts: {len(series_list.get_futures())}")
print(f"Options Contracts: {len(series_list.get_options())}")

# Expected output:
# Total Series: 250
# Active Series: 120
# Futures Contracts: 80
# Options Contracts: 170

### Understanding the Response

The `series_list` object contains:
- **series**: A list of all TFEX contracts
- **count**: Total number of series (property)

Each series has information like:
- Symbol (e.g., "S50Z25")
- Instrument name (e.g., "SET50 Futures")
- Contract expiration dates
- Underlying asset (e.g., "SET50")
- Whether it's active for trading

In [ ]:
# Let's look at the first few series
print("First 5 TFEX Series:\n")
for series in series_list.series[:5]:
    print(f"Symbol: {series.symbol}")
    print(f"  Name: {series.instrument_name}")
    print(f"  Underlying: {series.underlying}")
    print(f"  Contract Month: {series.contract_month}")
    print(f"  Active: {series.active}")
    print(f"  Expires: {series.last_trading_date.date()}")
    print()

# Expected output:
# Symbol: S50Z25
#   Name: SET50 Futures
#   Underlying: SET50
#   Contract Month: 12/2025
#   Active: True
#   Expires: 2025-12-29

## Advanced Usage - Filtering Contracts

The real power comes from filtering the series list to find exactly what you need.

### Filter Active Futures Only

Most traders only care about **active** contracts (those currently trading). Let's filter for active futures.

In [ ]:
# Get only active futures contracts
all_futures = series_list.get_futures()
active_futures = [s for s in all_futures if s.active]

print(f"Total Futures: {len(all_futures)}")
print(f"Active Futures: {len(active_futures)}")
print("\nActive Futures Contracts:\n")

for series in active_futures[:10]:  # Show first 10
    days_left = (series.last_trading_date - datetime.now(series.last_trading_date.tzinfo)).days
    print(f"{series.symbol:<12} {series.instrument_name:<30} Expires in {days_left:>3} days")

# Expected output:
# Total Futures: 80
# Active Futures: 35
#
# Active Futures Contracts:
#
# S50Z25       SET50 Futures                  Expires in  85 days
# S50V25       SET50 Futures                  Expires in  25 days

### Filter by Underlying Asset

Want to see all contracts for a specific underlying? Use `filter_by_underlying()`.

In [ ]:
# Get all SET50 contracts (both futures and options)
set50_series = series_list.filter_by_underlying("SET50")
set50_active = [s for s in set50_series if s.active]

print(f"Total SET50 Contracts: {len(set50_series)}")
print(f"Active SET50 Contracts: {len(set50_active)}")

# Separate futures and options
set50_futures = [s for s in set50_active if not s.options_type]
set50_options = [s for s in set50_active if s.options_type]

print(f"\nSET50 Futures: {len(set50_futures)}")
print(f"SET50 Options: {len(set50_options)}")

# Show futures sorted by expiration
print("\nSET50 Futures by Expiration:\n")
set50_futures.sort(key=lambda x: x.last_trading_date)
for series in set50_futures:
    print(f"{series.symbol:<12} {series.contract_month:<12} Expires: {series.last_trading_date.date()}")

# Expected output:
# Total SET50 Contracts: 85
# Active SET50 Contracts: 45
#
# SET50 Futures: 4
# SET50 Options: 41
#
# SET50 Futures by Expiration:
#
# S50V25       10/2025      Expires: 2025-10-30
# S50X25       11/2025      Expires: 2025-11-27
# S50Z25       12/2025      Expires: 2025-12-29

### Lookup Specific Series

If you know the symbol, you can look it up directly.

In [ ]:
# Lookup a specific series by symbol
symbol = "S50Z25"
series = series_list.get_symbol(symbol)

if series:
    print(f"Series Details for {symbol}:\n")
    print(f"  Instrument: {series.instrument_name}")
    print(f"  Market: {series.market_list_name}")
    print(f"  Underlying: {series.underlying}")
    print(f"  Contract Month: {series.contract_month}")
    print(f"  First Trading: {series.first_trading_date.date()}")
    print(f"  Last Trading: {series.last_trading_date.date()}")
    print(f"  Active: {series.active}")
    print(f"  Night Session: {series.has_night_session}")
    
    # Check if it's futures or options
    if series.options_type:
        opt_type = "Call" if series.options_type == "C" else "Put"
        print(f"  Type: Options ({opt_type})")
        print(f"  Strike Price: {series.strike_price}")
    else:
        print(f"  Type: Futures")
else:
    print(f"Series {symbol} not found!")

# Expected output:
# Series Details for S50Z25:
#
#   Instrument: SET50 Futures
#   Market: Equity Index Futures
#   Underlying: SET50
#   Contract Month: 12/2025
#   First Trading: 2025-09-30
#   Last Trading: 2025-12-29
#   Active: True
#   Night Session: False
#   Type: Futures

## Derivatives Trading Use Cases

Now let's explore real-world scenarios that derivatives traders face every day.

### Use Case 1: Contract Rollover Monitoring

**What is Contract Rollover?**

Futures contracts expire every month. Before your current contract expires, you need to "roll over" to the next month's contract. This involves:
1. Closing your position in the expiring contract ("front month")
2. Opening a position in the next contract ("next month")

**Why is this important?**

- Avoid forced settlement at expiration
- Maintain continuous market exposure
- Plan rollover timing to minimize cost ("roll cost")

Let's identify when to roll our SET50 futures position:

In [ ]:
# Find SET50 futures for rollover planning
set50_futures = series_list.filter_by_underlying("SET50")
set50_futures = [s for s in set50_futures if s.active and not s.options_type]

# Sort by expiration date
set50_futures.sort(key=lambda x: x.last_trading_date)

if len(set50_futures) >= 2:
    front_month = set50_futures[0]
    next_month = set50_futures[1]
    
    # Calculate days to expiration
    now = datetime.now(front_month.last_trading_date.tzinfo)
    days_to_expiry = (front_month.last_trading_date - now).days
    
    print("CONTRACT ROLLOVER PLAN\n")
    print("Front Month (Current Position):")
    print(f"  Symbol: {front_month.symbol}")
    print(f"  Expires: {front_month.last_trading_date.date()}")
    print(f"  Days Until Expiry: {days_to_expiry}")
    
    print("\nNext Month (Roll Target):")
    print(f"  Symbol: {next_month.symbol}")
    print(f"  Expires: {next_month.last_trading_date.date()}")
    
    # Rollover recommendation
    print("\nRecommendation:")
    if days_to_expiry <= 5:
        print(f"  🚨 URGENT: Roll immediately! Only {days_to_expiry} days left.")
    elif days_to_expiry <= 10:
        print(f"  ⚠️  PLAN ROLLOVER: Consider rolling in next few days.")
    else:
        print(f"  ✓ MONITOR: {days_to_expiry} days remaining. Plan rollover soon.")
    
    # Rollover steps
    print("\nRollover Steps:")
    print(f"  1. Close position in {front_month.symbol}")
    print(f"  2. Open position in {next_month.symbol}")
    print(f"  3. Monitor roll cost (price difference between contracts)")

# Expected output:
# CONTRACT ROLLOVER PLAN
#
# Front Month (Current Position):
#   Symbol: S50V25
#   Expires: 2025-10-30
#   Days Until Expiry: 25
#
# Next Month (Roll Target):
#   Symbol: S50X25
#   Expires: 2025-11-27
#
# Recommendation:
#   ✓ MONITOR: 25 days remaining. Plan rollover soon.

### Use Case 2: Options Chain Analysis

**What is an Options Chain?**

An options chain shows all available Call and Put options for an underlying asset, organized by:
- **Strike prices** - The price at which you can buy/sell the underlying
- **Expiration dates** - When the option expires
- **Call vs Put** - Right to buy vs right to sell

**Why analyze options chains?**

- Choose optimal strike prices for your strategy
- Compare different expiration dates
- Build complex options strategies (spreads, straddles, etc.)

Let's analyze the SET50 options chain:

In [ ]:
# Get all active SET50 options
set50_options = series_list.filter_by_underlying("SET50")
set50_options = [s for s in set50_options if s.active and s.options_type]

# Separate calls and puts
calls = [opt for opt in set50_options if opt.options_type == "C"]
puts = [opt for opt in set50_options if opt.options_type == "P"]

print("OPTIONS CHAIN ANALYSIS - SET50\n")
print(f"Total Active Options: {len(set50_options)}")
print(f"  Call Options: {len(calls)}")
print(f"  Put Options: {len(puts)}")

# Analyze by contract month
from collections import defaultdict

options_by_month = defaultdict(lambda: {'calls': [], 'puts': []})
for opt in set50_options:
    if opt.options_type == "C":
        options_by_month[opt.contract_month]['calls'].append(opt)
    else:
        options_by_month[opt.contract_month]['puts'].append(opt)

# Show options by expiration month
print("\nOptions by Expiration Month:\n")
for month in sorted(options_by_month.keys()):
    opts = options_by_month[month]
    print(f"{month}:")
    print(f"  Calls: {len(opts['calls'])}, Puts: {len(opts['puts'])}")
    
    # Show strike prices for calls
    if opts['calls']:
        strikes = sorted(set(c.strike_price for c in opts['calls']))
        print(f"  Strike Range: {strikes[0]:.0f} - {strikes[-1]:.0f}")

# Show sample options chain for nearest expiry
if options_by_month:
    nearest_month = sorted(options_by_month.keys())[0]
    opts = options_by_month[nearest_month]
    
    print(f"\nSample Options Chain - {nearest_month}:\n")
    print(f"{'Strike':<10} {'Call Symbol':<15} {'Put Symbol':<15}")
    print("-" * 40)
    
    # Get unique strike prices
    strikes = sorted(set(c.strike_price for c in opts['calls']))[:5]  # First 5
    
    for strike in strikes:
        call = next((c for c in opts['calls'] if c.strike_price == strike), None)
        put = next((p for p in opts['puts'] if p.strike_price == strike), None)
        
        call_symbol = call.symbol if call else "-"
        put_symbol = put.symbol if put else "-"
        
        print(f"{strike:<10.0f} {call_symbol:<15} {put_symbol:<15}")

# Expected output:
# OPTIONS CHAIN ANALYSIS - SET50
#
# Total Active Options: 42
#   Call Options: 21
#   Put Options: 21
#
# Options by Expiration Month:
#
# 10/2025:
#   Calls: 7, Puts: 7
#   Strike Range: 800 - 920
# 11/2025:
#   Calls: 7, Puts: 7
#   Strike Range: 800 - 920

### Use Case 3: Find Contracts Expiring Soon

**Why monitor near-expiry contracts?**

- **Risk management** - Avoid unexpected expiration
- **Liquidity concerns** - Volume often drops in final days
- **Time decay** - Options lose value rapidly near expiration
- **Settlement planning** - Prepare for cash or physical settlement

Let's find all contracts expiring in the next 30 days:

In [ ]:
# Find contracts expiring in next 30 days
active_series = series_list.filter_active_only()

now = datetime.now(active_series[0].last_trading_date.tzinfo)
thirty_days = now + timedelta(days=30)

near_expiry = [
    s for s in active_series
    if now <= s.last_trading_date <= thirty_days
]

# Sort by expiration date (soonest first)
near_expiry.sort(key=lambda x: x.last_trading_date)

print("CONTRACTS EXPIRING IN NEXT 30 DAYS\n")
print(f"Total: {len(near_expiry)} contracts\n")
print(f"{'Symbol':<12} {'Type':<12} {'Underlying':<12} {'Days Left':<12} {'Expiry Date'}")
print("-" * 70)

for series in near_expiry[:15]:  # Show first 15
    days_left = (series.last_trading_date - now).days
    
    # Determine type
    if series.options_type:
        contract_type = "Call" if series.options_type == "C" else "Put"
    else:
        contract_type = "Futures"
    
    # Alert level based on days left
    if days_left <= 7:
        alert = "🔴"  # Critical - Less than 1 week
    elif days_left <= 14:
        alert = "🟡"  # Warning - Less than 2 weeks
    else:
        alert = "🟢"  # Normal - More than 2 weeks
    
    print(f"{alert} {series.symbol:<10} {contract_type:<12} {series.underlying:<12} {days_left:<12} {series.last_trading_date.date()}")

# Action recommendations
critical = [s for s in near_expiry if (s.last_trading_date - now).days <= 7]
warning = [s for s in near_expiry if 7 < (s.last_trading_date - now).days <= 14]

print("\nACTION REQUIRED:\n")
if critical:
    print(f"🔴 CRITICAL: {len(critical)} contracts expire in ≤7 days - Roll or close immediately!")
if warning:
    print(f"🟡 WARNING: {len(warning)} contracts expire in 8-14 days - Plan rollover")
if not critical and not warning:
    print("🟢 All contracts have >14 days to expiration")

# Expected output:
# CONTRACTS EXPIRING IN NEXT 30 DAYS
#
# Total: 12 contracts
#
# Symbol       Type         Underlying   Days Left    Expiry Date
# ----------------------------------------------------------------------
# 🔴 S50V25      Futures      SET50        5            2025-10-30
# 🟡 S50V25C850  Call         SET50        12           2025-10-30
# 🟢 GOLDV25     Futures      GOLD         28           2025-10-25

### Use Case 4: Compare Underlying Assets

**What are TFEX Underlying Assets?**

TFEX derivatives are based on various underlying assets:
- **SET50** - Top 50 stocks by market cap on SET
- **GOLD** - Gold futures
- **USD** - US Dollar futures
- **Others** - Agricultural commodities, etc.

Let's analyze which underlyings have the most trading opportunities:

In [ ]:
# Analyze contract availability by underlying asset
active_series = series_list.filter_active_only()

# Group by underlying
by_underlying = defaultdict(lambda: {'futures': 0, 'options': 0, 'total': 0})

for series in active_series:
    underlying = series.underlying
    by_underlying[underlying]['total'] += 1
    
    if series.options_type:
        by_underlying[underlying]['options'] += 1
    else:
        by_underlying[underlying]['futures'] += 1

# Sort by total contracts (most to least)
sorted_underlyings = sorted(by_underlying.items(), key=lambda x: x[1]['total'], reverse=True)

print("ACTIVE CONTRACTS BY UNDERLYING ASSET\n")
print(f"{'Underlying':<15} {'Futures':<12} {'Options':<12} {'Total':<12}")
print("-" * 55)

for underlying, counts in sorted_underlyings:
    print(f"{underlying:<15} {counts['futures']:<12} {counts['options']:<12} {counts['total']:<12}")

# Trading opportunity analysis
print("\nTRADING OPPORTUNITY ANALYSIS:\n")

for underlying, counts in sorted_underlyings[:5]:  # Top 5
    print(f"{underlying}:")
    
    if counts['futures'] > 0 and counts['options'] > 0:
        print(f"  ✓ Both futures and options available")
        print(f"  ✓ Suitable for: Hedging, speculation, complex strategies")
    elif counts['futures'] > 0:
        print(f"  ✓ Futures only")
        print(f"  ✓ Suitable for: Directional trading, hedging")
    else:
        print(f"  ✓ Options only")
        print(f"  ✓ Suitable for: Limited risk strategies")
    
    print()

# Expected output:
# ACTIVE CONTRACTS BY UNDERLYING ASSET
#
# Underlying      Futures      Options      Total       
# -------------------------------------------------------
# SET50           4            38           42          
# GOLD            3            0            3           
# USD             2            0            2           
#
# TRADING OPPORTUNITY ANALYSIS:
#
# SET50:
#   ✓ Both futures and options available
#   ✓ Suitable for: Hedging, speculation, complex strategies

## Error Handling

Always handle errors gracefully when working with external APIs.

In [ ]:
# Safe fetch with error handling
async def safe_fetch_series_list():
    """
    Fetch TFEX series list with comprehensive error handling.
    
    Returns:
        TFEXSeriesListResponse or None if failed
    """
    try:
        series_list = await get_series_list()
        print(f"✓ Successfully fetched {series_list.count} series")
        return series_list
        
    except ConnectionError as e:
        print(f"✗ Network error: {e}")
        print("  Check your internet connection and try again.")
        return None
        
    except Exception as e:
        print(f"✗ Unexpected error: {e}")
        print("  Please report this issue.")
        return None

# Use the safe fetch function
series_list = await safe_fetch_series_list()

if series_list:
    print(f"\nTotal series: {series_list.count}")
    print(f"Active series: {len(series_list.filter_active_only())}")
else:
    print("\nFailed to fetch series list. Cannot proceed.")

# Expected output:
# ✓ Successfully fetched 250 series
#
# Total series: 250
# Active series: 120

In [ ]:
# Handle missing symbols gracefully
def lookup_series_safe(series_list, symbol):
    """
    Safely lookup a series by symbol.
    
    Args:
        series_list: TFEXSeriesListResponse object
        symbol: Series symbol to lookup
    
    Returns:
        TFEXSeries or None if not found
    """
    series = series_list.get_symbol(symbol)
    
    if series:
        print(f"✓ Found: {series.symbol} - {series.instrument_name}")
        return series
    else:
        print(f"✗ Series '{symbol}' not found in TFEX series list")
        print("  Tip: Check symbol spelling or use filter methods to explore available series")
        return None

# Test with valid and invalid symbols
print("Looking up S50Z25:")
series = lookup_series_safe(series_list, "S50Z25")

print("\nLooking up invalid symbol:")
series = lookup_series_safe(series_list, "INVALID123")

# Expected output:
# Looking up S50Z25:
# ✓ Found: S50Z25 - SET50 Futures
#
# Looking up invalid symbol:
# ✗ Series 'INVALID123' not found in TFEX series list
#   Tip: Check symbol spelling or use filter methods to explore available series

## Next Steps

Now that you know how to fetch and filter TFEX series, continue learning:

### Related Notebooks
- **02_trading_statistics.ipynb** - Get margin requirements, settlement prices, and days to maturity for individual contracts

### Documentation
- [TFEX Series List Service Docs](../../docs/settfex/services/tfex/list.md) - Complete API reference
- [TFEX Trading Statistics Docs](../../docs/settfex/services/tfex/trading_statistics.md) - Learn about trading statistics

### Key Takeaways

1. **Use `get_series_list()`** to fetch all TFEX contracts
2. **Filter methods** help you find exactly what you need:
   - `filter_active_only()` - Only trading contracts
   - `get_futures()` / `get_options()` - By contract type
   - `filter_by_underlying()` - By underlying asset
   - `get_symbol()` - Specific symbol lookup
3. **Monitor expiration dates** for rollover planning
4. **Options chains** show all strikes and expirations for an underlying
5. **Always handle errors** when working with APIs

### Real-World Applications

- **Automated trading bots** - Find contracts programmatically
- **Portfolio management** - Track all your positions
- **Research tools** - Analyze market structure
- **Alert systems** - Notify before contract expiration

Happy trading! 🚀